In [1]:
import pandas as pd

# Load the uploaded CSV file
file_path = '../../data/question_survey/questions_12_12_24_noemail.csv'
survey_data = pd.read_csv(file_path)

# Drop the 'email' column to be sure - ignore if not present
if 'email' in survey_data.columns:
    survey_data = survey_data.drop(columns='email')

# Display the first few rows of the DataFrame
survey_data.head()

,id,submitdate,lastpage,startlanguage,seed,age,gender,gender[other],educationlevel,educationstatus,...,q9Time,q10Time,q11Time,q12Time,q13Time,q14Time,q15Time,langcodeTime,groupTime95801,emailTime
0,15,NaN,NaN,de,1176674547,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16,1980-01-01 00:00:00,3.0,de,26659429,A2,A1,NaN,A4,A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.23,NaN
2,17,1980-01-01 00:00:00,3.0,en,1733042433,A2,A2,NaN,A4,A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.71,NaN
3,18,1980-01-01 00:00:00,3.0,de,1585812275,A2,A1,NaN,A4,A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.84,NaN
4,19,1980-01-01 00:00:00,3.0,de,380140469,A3,A2,NaN,A5,A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.35,NaN


In [2]:
# print rows and columns before dropping irrelevant columns
print("Before dropping irrelevant columns")
print(f'Rows: {survey_data.shape[0]}')
print(f'Columns: {survey_data.shape[1]}\n')

# Drop irrelevant columns directly to ensure anonymity and clean structure
columns_to_drop = ['email', 'submitdate', 'seed', 'startlanguage','ageTime','genderTime','educationlevelTime','educationstatusTime',
                   'programTime','familiarTime','q1Time','q2Time','q3Time','q4Time','q5Time','q6Time','q7Time','q8Time','q9Time','q10Time',
                   'q11Time','q12Time','q13Time','q14Time','q15Time','langcodeTime','emailTime']  # Columns we don't need
survey_data_clean = survey_data.drop(columns=[col for col in columns_to_drop if col in survey_data.columns], errors='ignore')

# print rows and columns after dropping irrelevant columns
print("After dropping irrelevant columns ")
print(f'Rows: {survey_data_clean.shape[0]}')
print(f'Columns: {survey_data_clean.shape[1]}\n')

# Drop rows where 'lastpage' is not 3.0 - survey not finished
survey_data_clean = survey_data_clean[survey_data['lastpage'] == 3.0]

# Drop lastpage column
survey_data_clean = survey_data_clean.drop(columns=['lastpage'])

# Print rows and column after dropping incomplete surveys
print("After dropping incomplete surveys ")
print(f'Rows: {survey_data_clean.shape[0]}')
print(f'Columns: {survey_data_clean.shape[1]}\n')

# Rename specific columns for readability
survey_data_clean = survey_data_clean.rename(columns={
    'groupTime95800': 'demographics_time',
    'groupTime95797': 'question_time',
    'groupTime95801': 'email_time',
    'langcode': 'question_language',
    'id': 'participant_id',
})

# Map education status to roles
role_mapping = {
    'A2': 'prospective',
    'A3': 'prospective',
    'A4': 'enrolled',
    'A5': 'international',
    'A1': 'other',
    'A6': 'other',
    'A7': 'other',
    'A8': 'other'
}
survey_data_clean['role'] = survey_data_clean['educationstatus'].map(role_mapping)

# Anomaly in participant_id == 37, did the survey in english but provided german questions
# Change language to german
survey_data_clean.loc[survey_data_clean['participant_id'] == 37, 'question_language'] = 'de'

# Transform questions into a long format, keeping all demographic information
question_columns = ['q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10', 'q11', 'q12', 'q13', 'q14', 'q15']
question_dataset = pd.melt(
    survey_data_clean,
    id_vars=[col for col in survey_data_clean.columns if col not in question_columns],  # Keep demographics
    value_vars=question_columns,  # Questions
    var_name='question_id',
    value_name='question_text'
)
# Drop question_id column
question_dataset = question_dataset.drop(columns=['question_id'])

# Drop rows with missing question text
question_dataset = question_dataset.dropna(subset=['question_text'])

# Add a column for unique question IDs
question_dataset['question_id'] = range(1, question_dataset.shape[0] + 1)

# Reorder columns for better readability
columns_order = (
    ['question_text', 'role', 'program', 'participant_id', 'question_id', 'question_language'] +
    [col for col in question_dataset.columns if col not in ['question_text', 'role', 'program', 'participant_id', 'question_id', 'question_language']]
)
question_dataset = question_dataset[columns_order]

# Print rows and column after long format transformation
print("After long format transformation")
print(f'Rows: {question_dataset.shape[0]}')
print(f'Columns: {question_dataset.shape[1]}\n')

# Print number of English and German questions per role
print("Number of English and German questions per role")
questions_per_role_language = question_dataset.groupby(['role', 'question_language']).size().unstack(fill_value=0)
print(questions_per_role_language)
# Print total englsi and german questions
print("Total English and German questions")
print(question_dataset['question_language'].value_counts())

# Save the cleaned question dataset to a CSV file
output_path = '../../data/question_survey/question_dataset_not_reviewed.csv'
question_dataset.to_csv(output_path, index=False, quoting=1)

question_dataset.head()


Before dropping irrelevant columns
Rows: 102
Columns: 55

After dropping irrelevant columns 
Rows: 102
Columns: 29

After dropping incomplete surveys 
Rows: 51
Columns: 28

After long format transformation
Rows: 370
Columns: 16

Number of English and German questions per role
question_language   de  en
role                      
enrolled           189  29
international        7  28
other               65   0
prospective         52   0
Total English and German questions
question_language
de    313
en     57
Name: count, dtype: int64


,question_text,role,program,participant_id,question_id,question_language,age,gender,gender[other],educationlevel,educationstatus,familiar,interviewtime,demographics_time,question_time,email_time
0,Wann kann ich mich für [STUDIENGANG] einschrei...,enrolled,Cognitive Science,16,1,de,A2,A1,NaN,A4,A4,A1,296.44,41.96,235.25,19.23
1,What do students usually learn in cognitive sc...,enrolled,Cognitive Science,17,2,en,A2,A2,NaN,A4,A4,A2,415.44,38.97,356.76,19.71
2,"Ich möchte hier studieren, wo kann ich mehr In...",enrolled,Bachelor of Science Cognitive Science,18,3,de,A2,A1,NaN,A4,A4,A2,539.12,66.19,456.09,16.84
3,Wie erreiche ich das Students Office?,enrolled,Cognitive Science,19,4,de,A3,A2,NaN,A5,A4,A1,419.18,34.73,379.10,5.35
4,Wie könnte mein Stundenplan im Studiengang Cog...,enrolled,Cognitive Science,20,5,de,A2,A2,NaN,A4,A4,A2,443.46,49.35,347.80,46.31


In [4]:
# filtering after personal review of questions

# Load the CSV file
file_path = '../../data/question_survey/question_dataset_not_reviewed.csv'
question_dataset = pd.read_csv(file_path)

# Filter out questions that are using a variable as a placeholder
# Questions with 'question_id' 1, 19, 29, 34, 43, 58, 66, 98, 99, 109, 182, 191, 205, 233, 256, 259, 335, 358, 369 (see question_review.xlsx)
question_dataset = question_dataset[~question_dataset['question_id'].isin([1, 19, 29, 34, 43, 58, 66, 98, 99, 109, 182, 191, 205, 233, 256, 259, 335, 358, 369])]

# Filter out questions that are no questions
# Questions with 'question_id' 235, 286, 303, 337, 354
question_dataset = question_dataset[~question_dataset['question_id'].isin([235, 286, 303, 337, 354])]

# Remove participant not taking survey seriously + logical error in demographic data (participant_id 98)
question_dataset = question_dataset[~question_dataset['participant_id'].isin([98])]

# Print rows and column after long format transformation
print("After long format transformation")
print(f'Rows: {question_dataset.shape[0]}')
print(f'Columns: {question_dataset.shape[1]}\n')

# Print number of English and German questions per role
print("Number of English and German questions per role")
questions_per_role_language = question_dataset.groupby(['role', 'question_language']).size().unstack(fill_value=0)
print(questions_per_role_language)
# Print total englsi and german questions
print("Total English and German questions")
print(question_dataset['question_language'].value_counts())

# Save the cleaned question dataset to a CSV file
output_path = '../../data/question_survey/question_dataset_reviewed.csv'
question_dataset.to_csv(output_path, index=False, quoting=1)


After long format transformation
Rows: 337
Columns: 16

Number of English and German questions per role
question_language   de  en
role                      
enrolled           174  29
international        7  26
other               49   0
prospective         52   0
Total English and German questions
question_language
de    282
en     55
Name: count, dtype: int64


In [1]:
# Translating questions between German <-> English
import os
import pandas as pd
from tqdm import tqdm
from openai import OpenAI

# Load environment variables
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

file_path = '../../data/question_survey/question_dataset_reviewed.csv'
question_data_translated = pd.read_csv(file_path)

def translate_text(text, target_language):
    """
    Translates a given text to the target language using GPT-4. 
    If there's an error, or if the target language isn't recognized, 
    return the original text and mark success=False.
    """
    try:
        if target_language == "de":
            prompt = f"Translate the following text to German:\n{text}"
        elif target_language == "en":
            prompt = f"Translate the following text to English:\n{text}"
        else:
            # If no recognized target language, return as-is
            return text, False

        system_message = {
            "role": "system",
            "content": (
                "You are a translation assistant. Please translate the user's input as accurately as possible, "
                "ensuring that the original meaning is preserved. Maintain all punctuation, formatting, and any "
                "special characters exactly as they appear in the original text. Do not add any additional information. "
                "OUTPUT ONLY THE TRANSLATED TEXT WITHOUT SPECIAL CHARACTERS BEFORE OR AFTER."
            )
        }

        response = client.chat.completions.create(
            messages=[system_message, {"role": "user", "content": prompt}],
            model="gpt-4o-2024-08-06",
            temperature=0,
        )

        translated = response.choices[0].message.content.strip()
        return translated, True

    except Exception as e:
        print(f"Error translating text: {e}")
        return text, False

# Prepare lists to store the final columns
german_questions = []
english_questions = []
translation_done = []

for _, row in tqdm(question_data_translated.iterrows(), total=question_data_translated.shape[0]):
    original_text = row['question_text']
    original_lang = row['question_language']

    # Default to storing the original text in both columns if something unexpected
    g_text = original_text
    e_text = original_text
    success = False

    # CASE 1: If original is English, store as English -> translate to German
    if original_lang == 'en':
        e_text = original_text
        g_text, success = translate_text(original_text, "de")
        print("Original text:\n", e_text)
        print("*" * 50)
        print("Translated text:\n", g_text)
        print("-" * 50)

    # CASE 2: If original is German, store as German -> translate to English
    elif original_lang == 'de':
        g_text = original_text
        e_text, success = translate_text(original_text, "en")
        print("Original text:\n", g_text)
        print("*" * 50)
        print("Translated text:\n", e_text)
        print("-" * 50)

    else:
        g_text = original_text
        e_text = original_text
        success = False

    german_questions.append(g_text)
    english_questions.append(e_text)
    translation_done.append(success)

# Insert the two new columns into the DataFrame
#   german_question_text  (index 0, i.e., first column)
#   english_question_text (index 1, i.e., second column)
question_data_translated.insert(0, 'german_question_text', german_questions)
question_data_translated.insert(1, 'english_question_text', english_questions)

# Indicate whether a translation took place
question_data_translated['translation_done'] = translation_done

# Append to every column '_q' to indicate that it is from the question survey
question_data_translated.columns = [f"{col}_q" for col in question_data_translated.columns]

# Finally, save to a new CSV
output_path_translated = '../../data/question_survey/questions_dataset_reviewed_translated_all.csv'
question_data_translated.to_csv(output_path_translated, index=False, quoting=1)

# Display the updated DataFrame
question_data_translated.head()


  0%|          | 1/337 [00:04<26:43,  4.77s/it]

Original text:
 What do students usually learn in cognitive science?
**************************************************
Translated text:
 Was lernen Studenten normalerweise in der Kognitionswissenschaft?
--------------------------------------------------


  1%|          | 2/337 [00:05<14:41,  2.63s/it]

Original text:
 Ich möchte hier studieren, wo kann ich mehr Infos über verschiedene Studiengänge finden?
**************************************************
Translated text:
 I want to study here, where can I find more information about different courses?
--------------------------------------------------


  1%|          | 3/337 [00:06<10:11,  1.83s/it]

...

100%|█████████▉| 336/337 [07:05<00:01,  1.34s/it]

Original text:
 Gibt es Besonderheiten für Psychologie in Osnabrück und in welche Richtungen kann ich mich später spezialisieren? 
**************************************************
Translated text:
 Are there any special features for psychology in Osnabrück and in which directions can I specialize later?
--------------------------------------------------


100%|██████████| 337/337 [07:09<00:00,  1.27s/it]

Original text:
 Wo kann ich sehen ab wann ich mich für Psychologie einschreiben kann? 
**************************************************
Translated text:
 Where can I see from when I can enroll for psychology?
--------------------------------------------------


,german_question_text_q,english_question_text_q,question_text_q,role_q,program_q,participant_id_q,question_id_q,question_language_q,age_q,gender_q,gender[other]_q,educationlevel_q,educationstatus_q,familiar_q,interviewtime_q,demographics_time_q,question_time_q,email_time_q,translation_done_q
0,Was lernen Studenten normalerweise in der Kogn...,What do students usually learn in cognitive sc...,What do students usually learn in cognitive sc...,enrolled,Cognitive Science,17,2,en,A2,A2,NaN,A4,A4,A2,415.44,38.97,356.76,19.71,True
1,"Ich möchte hier studieren, wo kann ich mehr In...","I want to study here, where can I find more in...","Ich möchte hier studieren, wo kann ich mehr In...",enrolled,Bachelor of Science Cognitive Science,18,3,de,A2,A1,NaN,A4,A4,A2,539.12,66.19,456.09,16.84,True
2,Wie erreiche ich das Students Office?,How do I reach the Students Office?,Wie erreiche ich das Students Office?,enrolled,Cognitive Science,19,4,de,A3,A2,NaN,A5,A4,A1,419.18,34.73,379.10,5.35,True
3,Wie könnte mein Stundenplan im Studiengang Cog...,What could my schedule look like in the Cognit...,Wie könnte mein Stundenplan im Studiengang Cog...,enrolled,Cognitive Science,20,5,de,A2,A2,NaN,A4,A4,A2,443.46,49.35,347.80,46.31,True
4,"Welche Studiengänge gibt es, die ausschließlic...",What courses are there that are taught exclusi...,"Welche Studiengänge gibt es, die ausschließlic...",enrolled,Cognitive Science,21,6,de,A2,A2,NaN,A5,A4,A3,848.80,55.92,751.32,41.56,True


In [3]:
file_path = '../../data/question_survey/questions_dataset_reviewed_translated_all.csv'
question_data_translated = pd.read_csv(file_path)
# drop the questoin_test_q column
question_data_translated = question_data_translated.drop(columns=['question_text_q'])
# Save the cleaned question dataset to a CSV file
output_path_translated = '../../data/question_survey/questions_dataset_reviewed_translated_de_en.csv'
question_data_translated.to_csv(output_path_translated, index=False, quoting=1)